In [15]:
import specpy

from autosted import AcquisitionPipeline
from autosted.imspector import get_current_stage_coords
from autosted.callback_buildingblocks.static_settings import JSONSettingsLoader
from autosted.callback_buildingblocks.parameter_filtering import LocationRemover
from autosted.callback_buildingblocks.regular_position_generators import SpiralOffsetGenerator
from autosted.stoppingcriteria import MaximumAcquisitionsStoppingCriterion
from autosted.taskgeneration import AcquisitionTaskGenerator
from autosted.callback_buildingblocks.stitched_data_selection import StitchedNewestDataSelector
from autosted.detection.roi_detection import ROIDetectorWrapper

In [16]:
# where to save & whether to save combined HDF5 file
save_folder = 'examples/acquisition_data/20241126_stitching_test'
save_hdf5 = True

# path of measurement parameters (dumped to JSON file)
measurement_parameters = 'examples/config_json/20241010_overview_3d_640.json'

# yx move size (should be smaller than FOV from parameters if we want on-the-fly stitching)
move_size = [45e-6, 45e-6]

In [ ]:
# get current coordinates and print, so we can go back to that position
start_coords = get_current_stage_coords()
print(start_coords)

In [ ]:
# build pipeline object (1 level: 'field')
pipeline = AcquisitionPipeline(save_folder, ['field'], save_combined_hdf5=save_hdf5, name='multipoint-acquisition')

# callback that will create an acquisition task with given measurement parameters
# at the next stage coordinates in the coordinate list (the next 'position')
next_position_generator = AcquisitionTaskGenerator('field',
    LocationRemover(JSONSettingsLoader(measurement_parameters)),
    SpiralOffsetGenerator(move_size, start_coords[1:])
    )

# a dummy detector with a detection function that always returns no detections
# it will, however, plot the stitched images with plot_detections=True (default)
dummy_callback = AcquisitionTaskGenerator('field',
    ROIDetectorWrapper(
        lambda x: [],
        data_source_callback=StitchedNewestDataSelector(pipeline, 'field', register_tiles=True),
        plot_detections=True)
)
# attach callback so that after each position, the next one will be enqueued
pipeline.add_callback(next_position_generator, 'field')

pipeline.add_callback(dummy_callback, 'field')

# set maximum number of acquisitions before stop
pipeline.add_stopping_condition(MaximumAcquisitionsStoppingCriterion(10))

# start with initial task from callback
pipeline.run(next_position_generator)